# Part 3: Logistic Regression with GloVe Embeddings

We first load in the dataset with our processed examples

In [ ]:
#imports

import csv
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import LogisticRegression

In [3]:
file = 'limited_theorem_classes_2.csv'

#Our X data is theorem statements in Lean, while categories are labels
categories = []
theorems = []


with open(file, mode='r', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)  # first row is column labels
    for row in csvreader:
        categories.append(row[0])
        theorems.append(row[1])

In [6]:
# We use Keras' inbuilt tokenizer to map unique words/special characters to unique integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(theorems)
sequences = tokenizer.texts_to_sequences(theorems)

Now, we load in GloVe embeddings from pretrained for us. GloVe is large and isn't submitted to gradescope. Ensure embeddings are downloaded prior to running. 

In [7]:
glove_file = 'glove/glove.6B.50d.txt'

#Standard load of glove embeddings from given data
def load_glove_embeddings(glove_file):
    embeddings = {}
    with open(glove_file, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vec = np.asarray(values[1:], "float32")
            embeddings[word] = vec
    return embeddings

glove_embeddings = load_glove_embeddings(glove_file)

For each lean statement in our data, we find the respective embedding

In [8]:
def text_to_embedding(sequence, embeddings_dict):
    vectors = [embeddings_dict.get(token, np.zeros(100)) for token in sequence]
    if vectors:
        return np.mean(vectors, axis=0) #mean pool all word embeddings across lean statement
    else:
        return np.zeros(100)

X_embeddings = np.array([text_to_embedding(sequence, glove_embeddings) for sequence in sequences])

To maintain equal vector dimenstions, we pad all lean statements to be the same size as the biggest. This uses Keras' inbuilt function for padding

In [59]:
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
print(max_length)

234


We one-hot encode all math categories

In [9]:
categories_index = {category: i for i, category in enumerate(set(categories))}
encoded_categories = [categories_index[category] for category in categories]
categorical_labels = to_categorical(encoded_categories)
print(len(categorical_labels[0]))

10


In [15]:
y_int_labels = np.argmax(categorical_labels, axis=1) #Convert one-hot into integer from 0-28


X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y_int_labels, test_size=0.2, random_state=229) #80-20 split

print(X_embeddings.shape)
print(categorical_labels.shape)

#Use sklearn's multinomial regression
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)


result = model.score(X_test, y_test)
print(f"Accuracy: {result}")

(77834, 100)
(77834, 10)
Accuracy: 0.2633134194128605
